In [1]:
import pandas as pd
import numpy as np
import json
import re
import os
from google.cloud import storage

In [2]:
#Bring data from GCS
os.environ['GOOGLE_APPLICATION_CREDENTIALS']="/Users/ryanchoi/Desktop/SADA U/Project/gmail output/storage_secret.json"

client = storage.Client()

In [3]:
#download from GCS
def download_blob(bucket_name, source_blob_name, destination_file_name):
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)

In [4]:
download_blob('rube_goldberg_project', 'reddit_data/redditHot.json', "hot.json")
download_blob('rube_goldberg_project', 'reddit_data/redditTop.json', "top.json")

In [5]:
#change to dataframe
hot=pd.read_json('hot.json', orient='records')
top=pd.read_json('top.json', orient='records')

ValueError: Trailing data

In [6]:
def clean(body):
    body = re.sub(r'\n', ' ', body)
    body = re.sub(r'http\S+', ' ', body)
    body = re.sub(r'&#x200B', ' ', body)
    body = re.sub(r"[^a-zA-Z'0-9]+", ' ', body)
    return body

In [7]:
hot['body']=hot['body'].apply(clean)
top['body']=top['body'].apply(clean)

NameError: name 'hot' is not defined

In [8]:
#blank body to nan
hot['body'].replace(['',' '], np.nan, inplace=True)
top['body'].replace(['',' '], np.nan, inplace=True)

NameError: name 'hot' is not defined

In [9]:
#Drop Nan
hot.dropna(subset = ["body"], inplace=True)
top.dropna(subset = ["body"], inplace=True)

NameError: name 'hot' is not defined

In [10]:
hot= hot.reset_index(drop=True)
top= top.reset_index(drop=True)

NameError: name 'hot' is not defined

In [11]:
# hot

In [12]:
from textblob import TextBlob
from textblob import Word
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [13]:
def getSubjectivity(body):
    return TextBlob(body).sentiment.subjectivity
def getPolarity(body):
    return TextBlob(body).sentiment.polarity

In [14]:
hot['subjectivity']=hot['body'].apply(getSubjectivity)
hot['Polarity']=hot['body'].apply(getPolarity)
top['subjectivity']=top['body'].apply(getSubjectivity)
top['Polarity']=top['body'].apply(getPolarity)

NameError: name 'hot' is not defined

In [ ]:
def getSentiment(polarity):
    if polarity < 0:
        return 'negative'
    elif polarity == 0:
        return 'neutral'
    else:
        return'positive'

In [ ]:
hot['Sentiment_Result']=hot['Polarity'].apply(getSentiment)
top['Sentiment_Result']=top['Polarity'].apply(getSentiment)

In [ ]:
hot['Category'] = 'Hot_Topic'
top['Category'] = 'Top_Topic'

In [ ]:
merge = pd.concat([hot,top])

In [ ]:
# merge

In [ ]:
# adding google NL merge into reddit df-------------------------------------------------------------------------

In [ ]:
download_blob('rube_goldberg_project', 'reddit_data/redditHot_analyzed.json', "hot_NL.json")
download_blob('rube_goldberg_project', 'reddit_data/redditTop_analyzed.json', "top_NL.json")

In [ ]:
with open('hot_NL.json','r') as f:
    hot_NL = json.loads(f.read())
    
with open('top_NL.json','r') as f:
    top_NL = json.loads(f.read())
    
top_NL_1 = pd.json_normalize(top_NL)
hot_NL_1 = pd.json_normalize(hot_NL)

In [ ]:
del top_NL_1['entity']
del hot_NL_1['entity']

In [ ]:
merge_NL = pd.concat([hot_NL_1,top_NL_1])

In [ ]:
merge_NL['sentiment.parsedSentiment'] = merge_NL['sentiment.parsedSentiment'].replace(['mixed'],'neutral')

In [15]:
# merge_NL

In [16]:
reddit = pd.merge(merge, merge_NL,how="left", on="id")

NameError: name 'merge' is not defined

In [17]:
# reddit

In [18]:
reddit=reddit.drop_duplicates()

NameError: name 'reddit' is not defined

In [19]:
def Check(df):
    if df['sentiment.parsedSentiment']== df['Sentiment_Result']:
        return "True"
    else:
        return "False" 

reddit['match'] = reddit.apply(Check, axis=1)
# reddit

NameError: name 'reddit' is not defined

In [20]:
#entity analysis

In [21]:
hot_NL_2 = pd.json_normalize(hot_NL, record_path =['entity'], meta = ['id',['sentiment','parsedSentiment'],['sentiment','score']])

top_NL_2 = pd.json_normalize(top_NL, record_path =['entity'], meta = ['id',['sentiment','parsedSentiment'],['sentiment','score']])

NameError: name 'hot_NL' is not defined

In [22]:
merge_NL_2 = pd.concat([hot_NL_2,top_NL_2])
merge_NL_2['sentiment.parsedSentiment'] = merge_NL_2['sentiment.parsedSentiment'].replace(['mixed'],'neutral')

NameError: name 'hot_NL_2' is not defined

In [23]:
merge_NL_2=merge_NL_2.drop_duplicates()

NameError: name 'merge_NL_2' is not defined

In [24]:
merge_NL_2['keyword']=merge_NL_2['keyword'].apply(clean)
merge_NL_2['keyword'].replace(['',' '], np.nan, inplace=True)

NameError: name 'merge_NL_2' is not defined

In [25]:
merge_NL_2.dropna(subset = ["keyword"], inplace=True)

NameError: name 'merge_NL_2' is not defined

In [26]:
#merge_NL_2

In [27]:
reddit_en = pd.merge(merge_NL_2, reddit[["id","subjectivity","Polarity", "Sentiment_Result", "Category", "match"]], how="left", on="id")

NameError: name 'merge_NL_2' is not defined

In [28]:
reddit_en.drop_duplicates(subset=['keyword','id'], keep="last", inplace=True)

NameError: name 'reddit_en' is not defined

In [29]:
reddit_en.to_csv('reddit_entity.csv')

NameError: name 'reddit_en' is not defined

In [30]:
reddit.to_csv('reddit_sentiment.csv')

NameError: name 'reddit' is not defined

In [31]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)


In [32]:
upload_blob('rube_goldberg_project', 'reddit_sentiment.csv', "reddit_data/reddit_Sentiment.csv")
upload_blob('rube_goldberg_project', 'reddit_entity.csv', "reddit_data/reddit_entity.csv")